![IE](../img/ie.png)

# Sessions 3 & 4: Project layout and unit tests

### Juan Luis Cano Rodríguez <jcano@faculty.ie.edu> - Master in Business Analytics and Big Data (2019-04-03)

## Triangular workflows in git

When collaborating with a project hosted online on GitHub or GitLab, the most common setup is having a central repository, one remote fork per user, and local clones/checkouts:

![Triangular workflow](https://github.blog/wp-content/uploads/2015/07/5dcdcae4-354a-11e5-9f82-915914fad4f7.png?resize=2000%2C951)

(Source: https://github.blog/2015-07-29-git-2-5-including-multiple-worktrees-and-triangular-workflows/)

Following this workflow requires discipline and sticking to a subset of actions and git commands to avoid common mistakes. This website contains all you need to know to setup your triangular workflow and we don't need to reproduce it here:

https://www.asmeurer.com/git-workflow/

*Notice* the different naming conventions between this website and the first image:

1. **Convention 1**: upstream/origin/local
2. **Convention 2**: origin/&#x3C;username&#x3E;/local

We will be consistent with the Aaron Meurer guide and therefore use Convention 2 all the time.

### ⚠ After creating a pull request ⚠

After your pull request has been merged to `master`, your local `master` and `<username>/master` will be outdated with respect to `origin/master`. On the other hand, **you should avoid working on this branch anymore in the future**: remember branches should be ephemeral and short-lived.

To put yourself in a clean state again, you have to:

1. Click "remove branch" in the pull request (don't click "remove fork"!)
2. `git checkout master` (go back to `master`)
3. `git fetch origin` (**never, ever use `git pull` unless you know exactly what you're doing**)
4. `git merge --ff-only origin master` (update your local `master` with `origin/master`, and fail if you accidentally made any commit in `master`)
5. `git fetch -p <username>` (✨ acknowledge the removal of the remote branch ✨)
6. `git branch -d old-branch` (remove the old branch)
7. `git push <username> master` (update your fork with respect to `origin`)
8. `git checkout -b new-branch` (start working in the new feature!)

This process **has to be repeated after every pull request**.

🌈 

## Project layout

Most data science projects will start with a bunch of notebooks. However, at some point we will want to reuse code between them, and eventually put our models in production without the need to use the notebooks themselves ([unless you are Netflix](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233)). Choosing a good project layout is extremely important to organize the code, avoid common pitfalls and be predictable (i.e. imitate the rest of the ecosystem to minimize surprise). On the other hand, there is lots (**lots**) of outdated, bad or wrong advice on the Internet about this topic, so here we will present The Truth™.

### References

* Packaging a Python library https://blog.ionelmc.ro/2014/05/25/python-packaging/
* Less known packaging features and tricks https://blog.ionelmc.ro/presentations/packaging/
* setuptools documentation https://setuptools.readthedocs.io/en/stable/setuptools.html

### The `src` layout

```
├─ src
│  └─ packagename
│     ├─ __init__.py
│     └─ ...
├─ tests
│  └─ ...
├─ README.txt
├─ setup.py
└─ setup.cfg
```

* The `src/packagename` contains the source code of the library.
  - The `packagename` is what users type after `import` in a Python script, and therefore should not contain special characters.
  - It should contain a `__init__.py` that can be empty (more on that below).
  - The `src` segment prevents you from *shooting yourself in the foot*, because it's common to do `import packagename` when you are developing, and this will import the code from the directory, not from your `sys.path`. Always include it.

* The `tests` directory contains the tests. It **must not** contain any `__init__.py` because it's not meant to be imported as a package. In very specific cases it's included inside `src/packagename`.

* Every project contains a `README.txt` that at least explains what the project is.

* `setup.py` can be an extremely simple file containing only this:

```
from setuptools import setup

setup()
```

(This requires `setuptools > 30.3.0`, released 8 Dec 2016)

* `setup.cfg` contains the metadata of the project. The absolutely required fields are `name`, `version`, and `packages`, therefore you will need something like this:

```
[metadata]
name = my_package
version = 0.1.dev0

# Magic! Don't touch below this line
[options]
package_dir=
    =src
packages=find:

[options.packages.find]
where=src
```

The `name` is what users will have to type after `pip install` and therefore can contain hyphens. **Do not confuse this** with what users have to type on `import` (see above).

With this layout, **you can `pip install` your code** in your Python environment:

```
$ pip install --editable .
$ python
>>> import packagename
>>>
```

This is an alternative to modifying the `PYTHONPATH` environment variable (see first session).

### Version numbers

* Version numbers for Python packages are explained in [PEP 440](https://www.python.org/dev/peps/pep-0440/)
* For libraries, the most widely used convention is [semantic versioning](https://semver.org/): `X.Y.Z`
  - `Z` **must** be incremented if only backwards compatible bug fixes are introduced (a bug fix is defined as an internal change that fixes incorrect behavior)
  - `Y` **must** be incremented every time there is new, backwards-compatible functionality
  - `X` **must** be incremented every time there are backwards-incompatible changes
* Between releases, the version should have the `.dev0` suffix
* Recommendation: start with `0.1.dev0` (development version), then make a `0.1.0` release, then progress to `0.1.1` for quick fixes and `0.2.0` for new functionality, and when you want to make a promise of *relative* stability jump to `1.0.0`
* For applications, other conventions are more appropriate, like [calendar versioning](http://calver.org/): `[YY]YY.MM.??

### Exercise

1. Create a directory called `ie-nlp-utils`
2. `git init` inside it
3. Create a basic `src` layout in it, with `name = ie-nlp-utils` and the source in `src/ie_nlp_utils`
4. Create a `src/ie_nlp_utils/__init__.py` with a `print("Hello, world!")`
5. Install it in editable mode using `pip` and test that `>>> import ie_nlp_utils` prints `Hello, world!`
6. Include a `README.txt` and an appropriate `.gitignore` from http://gitignore.io/
7. Commit the changes
8. Create a new GitHub project and push the repository there

🎉

## Testing

Testing is **essential**. Many developers get along without testing their software, but as common wisdom says:

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">If you use software that lacks automated tests, you are the tests.</p>&mdash; Jenny Bryan (@JennyBryan) <a href="https://twitter.com/JennyBryan/status/1043307291909316609?ref_src=twsrc%5Etfw">September 22, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Computers excel at doing repetitive tasks: they basically never make mistakes (the mistake might be in what we told the computer to do). Humans, on the other hand, fail more often, especially under pressure, or on Friday afternoons and Monday mornings. Therefore, instead of letting the humans be the tests, we will use the computer to **frequently verify that our software works as specified**.

### References

* pytest documentation https://docs.pytest.org

### Further reading

* Extreme Programming https://www.wikiwand.com/en/Extreme_programming
* Obey the Testing Goat! http://www.obeythetestinggoat.com/pages/book.html#toc
* (Shameless self-plug) Testing and validation approaches for scientific software https://nbviewer.jupyter.org/format/slides/github/poliastro/oscw2018-talk/blob/master/Talk.ipynb

### Test-Driven Development

> Make it work. Make it right. Make it fast.

Test-Driven Development shifts the focus of software development to writing tests. The "practice of test-first development, planning and writing tests before each micro-increment" is not new: it was in use at NASA in the early 1960s ([source](https://www.wikiwand.com/en/Extreme_programming)). In the 1990s, Extreme Programming took this concept to the extreme by the use of **small, automated** tests.

The "test-driven development mantra" is <span style="color: red">**Red**</span> - <span style="color: green">**Green**</span> - **Refactor**:

![The mantra](../img/red-green-refactor.png)

1. Write a test. <span style="color: red">**Watch it fail**</span>.
2. Write just enough code to <span style="color: green">**pass the test**</span>.
3. Improve the code without breaking the test.
4. Repeat.

### Testing in Python

Summary: **use pytest**. Everybody does. It rocks.

[pytest](https://docs.pytest.org/) is a testing framework for Python that makes writing tests extremely easy. It is much more powerful than the standard library equivalent, `unittest`. To use it, you need to install it first:

```
$ pip install pytest
```

The simplest test is **a function with an `assert`**. The `assert` statement just fails if the contents are not `True`, and else it does nothing. *It should only be used for testing*.

In [1]:
assert True  # Does nothing

In [2]:
assert False  # Fails!

AssertionError: 

In [3]:
assert 2 + 2 == 5, "Math is wrong"  # Fails with a message

AssertionError: Math is wrong

### Example

> Write a function that **tokenizes a sentence** (i.e. splits it into a list of words)

First, we write a (failing) test:

```python
# tests/test_tokenize.py
from ie_nlp_utils import tokenize  # This will fail right away!

def test_tokenize_returns_expected_list():
    sentence = "This is a sentence"
    expected_tokens = ["This", "is", "a", "sentence"]

    tokens = tokenize(sentence)

    assert tokens == expected_tokens
```

and we run it from the command line:

```
$ pytest
...
```

Then we fix the test in the simplest way:

```python
# src/nlp_utils/__init__.py
def tokenize(sentence):
    return sentence.split()
```

And we watch it pass!

```
$ pytest
...
```

### Exercise

1. Add a new test that checks that `tokenize(sentence, lower=True)` returns a list of *lowercase* tokens.
2. Fix the test *in a way the first one doesn't break*.
3. *Extra*: Use `@pytest.mark.parametrize` to pass two different sentences to the new test https://docs.pytest.org/en/latest/example/parametrize.html